## **Computer assignment 2: Routing**

**Submitted by**

Student 1: Sude Yiǧit

Student 2: Loïs Jonathas

1
---

**Imports and data reading**

In [2]:
import pandas as pd
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

# Reading in the data
def read_data():
    store_data = pd.read_csv('data/storedata.csv')
    travel_time = pd.read_csv('data/storeTraveltime.csv', index_col=0)
    travel_cost = pd.read_csv('data/storeTravelcost.csv', index_col=0)
    return store_data, travel_time, travel_cost

# Storing the data
store_data, travel_time, travel_cost = read_data()


**Helper Functions**

In [3]:
# Extracting route from OR-Tools solution
def extract_solution(manager, routing, solution):
    index = routing.Start(0)
    route = [manager.IndexToNode(index)]
    
    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
    
    return route

# Extracting all routes from OR-Tools solution for multiple vehicles
def extract_routes(manager, routing, solution, num_vehicles):
    routes = []
    for vehicle_id in range(num_vehicles):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
    return routes            

# Calculating the total time and cost for a route
def calculate_route_stats(route, travel_time_matrix, travel_cost_matrix):
    total_time = 0
    total_cost = 0
    
    for i in range(len(route) - 1):
        from_node = route[i]
        to_node = route[i + 1]
        total_time += travel_time_matrix.iloc[from_node, to_node]
        total_cost += travel_cost_matrix.iloc[from_node, to_node]
    
    return total_time, total_cost

# Calculating the total cost for all routes
def calculate_total_cost(routes, travel_cost_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i + 1]
            total_cost += travel_cost_matrix.iloc[from_node, to_node]
    return total_cost

##### **a) **

Since this is a Traveling Salesman Problem (TSP), we'll be using a heuristic approach with OR-Toools to find a good (near-optimal) route. The `storeTraveltime.csv` file will serve as cost matrix for the route optimization, while the `storeTravelcost.csv` will be used to compute the final total cost. 

In [4]:
def solve_tsp_ortools(travel_time):
    def create_data_model(travel_time):
        data = {}
        time_matrix = travel_time.values.astype(int).tolist() # Converted to integer for OR-tools
        data['time_matrix'] = time_matrix
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data
    data = create_data_model(travel_time)
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager) # The routing model

    # Creating and registering a transit callback
    def time_callback(from_index, to_index):
        # Returning the time between the two nodes
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node] 
    
    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    # Defining the cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

       # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = 30  
    
    # Solving the problem
    solution = routing.SolveWithParameters(search_parameters)
    
    # Extracting the solution
    if solution:
        route = extract_solution(manager, routing, solution)
        return route
    else:
        print("No solution found!")
        return None

# Solving the problem and displaying the solution 
print("Problem 1a: TSP Solution")
route_1a = solve_tsp_ortools(travel_time)
if route_1a:
        total_time, total_cost = calculate_route_stats(route_1a, travel_time, travel_cost)
        print(f"Total driving time: {total_time} minutes")
        print(f"Total cost: €{total_cost:.2f}")
        print(f"Number of locations visited: {len(route_1a)-1}")  # -1 because depot appears twice
        print(f"Route: {route_1a}")
        
        # Saving the solution to file
        with open('1a.txt', 'w') as f:
            f.write(' '.join(map(str, route_1a)))


Problem 1a: TSP Solution
Total driving time: 1297 minutes
Total cost: €1765.00
Number of locations visited: 50
Route: [0, 13, 31, 26, 27, 40, 25, 48, 37, 11, 46, 49, 39, 42, 29, 15, 45, 22, 36, 43, 3, 1, 19, 38, 32, 35, 34, 28, 6, 20, 5, 4, 9, 8, 21, 41, 7, 47, 24, 2, 44, 14, 30, 17, 18, 33, 12, 10, 16, 23, 0]


In [5]:
# Problem 1b - VRPTW (multiple salesmen with time windows)
def solve_vrptw_ortools(store_data, travel_time, travel_cost):
    def create_data_model(store_data, travel_time, travel_cost, num_vehicles):
        data = {}
        data['time_matrix'] = travel_time.values.astype(int).tolist()
        data['cost_matrix'] = travel_cost.values.astype(int).tolist()
        data['time_windows'] = []
        data['service_time'] = 15
        
        # Add depot time window (8 hours werkdag - 480 minuten)
        data['time_windows'].append((0, 480))
        
        # Add store time windows
        for _, store in store_data.iterrows():
            open_time = int(store['OpeningTime'])
            close_time = int(store['ClosingTime'])
            data['time_windows'].append((open_time, close_time))
        
        data['num_vehicles'] = num_vehicles
        data['depot'] = 0
        data['vehicle_capacity'] = 480  # 8 uur in minuten
        return data
    
    max_vehicles = len(store_data)
    best_solution = None
    best_cost = float('inf')
    best_num_vehicles = max_vehicles
    
    print(f"Total stores to visit: {len(store_data)}")
    print(f"Maximum vehicles to try: {max_vehicles}")
    
    # We gaan van weinig naar veel voertuigen om de minimale oplossing te vinden
    for num_vehicles in range(1, max_vehicles + 1):
        print(f"\nTrying with {num_vehicles} vehicles...")
        
        try:
            data = create_data_model(store_data, travel_time, travel_cost, num_vehicles)
            
            manager = pywrapcp.RoutingIndexManager(
                len(data['time_matrix']), 
                data['num_vehicles'], 
                data['depot']
            )
            routing = pywrapcp.RoutingModel(manager)
            
            def time_callback(from_index, to_index):
                from_node = manager.IndexToNode(from_index)
                to_node = manager.IndexToNode(to_index)
                travel_time_val = data['time_matrix'][from_node][to_node]
                # Service time alleen toevoegen als we van een store vertrekken (niet depot)
                service_time = data['service_time'] if from_node != data['depot'] else 0
                return travel_time_val + service_time
            
            def cost_callback(from_index, to_index):
                from_node = manager.IndexToNode(from_index)
                to_node = manager.IndexToNode(to_index)
                return data['cost_matrix'][from_node][to_node]
            
            transit_callback_index = routing.RegisterTransitCallback(time_callback)
            cost_callback_index = routing.RegisterTransitCallback(cost_callback)
            routing.SetArcCostEvaluatorOfAllVehicles(cost_callback_index)
            
            # Add time dimension
            routing.AddDimension(
                transit_callback_index,
                60,  # allow waiting time
                data['vehicle_capacity'],  # maximum time per vehicle
                False,  # Don't force start cumul to zero
                'Time'
            )
            time_dimension = routing.GetDimensionOrDie('Time')
            
            # Add time window constraints for all locations
            for location_idx in range(len(data['time_windows'])):
                index = manager.NodeToIndex(location_idx)
                time_window = data['time_windows'][location_idx]
                time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
            
            # Set depot start and end times
            depot_time_window = data['time_windows'][0]
            for vehicle_id in range(data['num_vehicles']):
                # Start at depot
                index = routing.Start(vehicle_id)
                time_dimension.CumulVar(index).SetRange(depot_time_window[0], depot_time_window[1])
                # End at depot  
                index = routing.End(vehicle_id)
                time_dimension.CumulVar(index).SetRange(depot_time_window[0], depot_time_window[1])
            
            # Force all locations to be visited
            for node in range(1, len(data['time_matrix'])):
                routing.AddDisjunction([manager.NodeToIndex(node)], 0)
            
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.first_solution_strategy = (
                routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
            )
            search_parameters.local_search_metaheuristic = (
                routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
            )
            search_parameters.time_limit.seconds = 10  # Korter voor snellere iteratie
            
            print(f"  Solving...")
            solution = routing.SolveWithParameters(search_parameters)
            
            if solution:
                routes = extract_routes(manager, routing, solution, data['num_vehicles'])
                total_cost = calculate_total_cost(routes, travel_cost)
                
                print(f"  ✓ Feasible solution found with {num_vehicles} vehicles, cost: €{total_cost:.2f}")
                
                # Check if all stores are visited
                visited_stores = set()
                for route in routes:
                    visited_stores.update(route)
                visited_stores.discard(0)  # remove depot
                
                if len(visited_stores) == len(store_data):
                    print(f"  ✓ All {len(store_data)} stores visited!")
                    best_solution = routes
                    best_cost = total_cost
                    best_num_vehicles = num_vehicles
                    print(f"  🎯 Found minimum vehicles: {best_num_vehicles}")
                    break  # Stop searching, we found the minimum
                else:
                    print(f"  ⚠ Only {len(visited_stores)}/{len(store_data)} stores visited, continuing...")
            else:
                print(f"  ✗ No solution with {num_vehicles} vehicles")
                
        except Exception as e:
            print(f"  ✗ Error with {num_vehicles} vehicles: {e}")
            continue
    
    if best_solution:
        print(f"\n🎯 BEST SOLUTION FOUND: {best_num_vehicles} vehicles with cost €{best_cost:.2f}")
        return best_solution, best_cost, best_num_vehicles
    else:
        print("\n❌ No feasible solution found with any number of vehicles!")
        print("Possible issues:")
        print("- Time windows too restrictive")
        print("- Vehicle capacity too low") 
        print("- Travel times too long")
        return None, None, None

# Eerst even controleren of onze helper functions werken
print("Testing helper functions...")

def extract_routes(manager, routing, solution, num_vehicles):
    """Extract all routes from OR-Tools solution"""
    routes = []
    for vehicle_id in range(num_vehicles):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))  # Add depot at end
        routes.append(route)
    return routes

def calculate_total_cost(routes, travel_cost_matrix):
    """Calculate total cost for all routes"""
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i + 1]
            total_cost += travel_cost_matrix.iloc[from_node, to_node]
    return total_cost

def calculate_route_stats(route, travel_time_matrix, travel_cost_matrix):
    """Calculate total time and cost for a route"""
    total_time = 0
    total_cost = 0
    for i in range(len(route) - 1):
        from_node = route[i]
        to_node = route[i + 1]
        total_time += travel_time_matrix.iloc[from_node, to_node]
        total_cost += travel_cost_matrix.iloc[from_node, to_node]
    return total_time, total_cost

print("Helper functions loaded successfully!")

# Solve and display results for 1b
print("\n" + "="*50)
print("Problem 1b - VRPTW Solution")
print("="*50)

routes_1b, total_cost_1b, num_vehicles_1b = solve_vrptw_ortools(store_data, travel_time, travel_cost)

if routes_1b:
    print(f"\n" + "="*50)
    print("FINAL SOLUTION")
    print("="*50)
    print(f"Minimum number of salesmen needed: {num_vehicles_1b}")
    print(f"Total cost: €{total_cost_1b:.2f}")
    print(f"\nIndividual routes:")
    
    for i, route in enumerate(routes_1b):
        route_time, route_cost = calculate_route_stats(route, travel_time, travel_cost)
        # Calculate actual time including service time
        service_time = (len(route) - 2) * 15  # -2 because depot at start and end
        actual_route_time = route_time + service_time
        print(f"Salesman {i+1}: {route}")
        print(f"  Travel time: {route_time} min, Service time: {service_time} min")
        print(f"  Total time: {actual_route_time} min, Cost: €{route_cost:.2f}")
        print()
    
    # Save solution
    with open('1b.txt', 'w') as f:
        for route in routes_1b:
            f.write(' '.join(map(str, route)) + '\n')
    print("Solution saved to 1b.txt")
else:
    print("No solution found. Let's try with relaxed constraints...")
    
    # Try with 24-hour capacity as fallback
    print("\nTrying with 24-hour capacity (1440 minutes)...")
    
    def solve_with_24h_capacity():
        # Simplified version with 24-hour capacity
        from ortools.constraint_solver import pywrapcp, routing_enums_pb2
        
        time_matrix = travel_time.values.astype(int).tolist()
        cost_matrix = travel_cost.values.astype(int).tolist()
        
        # Try different number of vehicles
        for num_vehicles in range(1, len(store_data) + 1):
            print(f"Trying {num_vehicles} vehicles with 24h capacity...")
            
            manager = pywrapcp.RoutingIndexManager(len(time_matrix), num_vehicles, 0)
            routing = pywrapcp.RoutingModel(manager)
            
            def time_callback(from_index, to_index):
                from_node = manager.IndexToNode(from_index)
                to_node = manager.IndexToNode(to_index)
                return time_matrix[from_node][to_node] + (15 if from_node != 0 else 0)
            
            transit_callback_index = routing.RegisterTransitCallback(time_callback)
            routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
            
            routing.AddDimension(
                transit_callback_index,
                1440,  # large waiting time
                1440,  # 24-hour capacity
                False,
                'Time'
            )
            
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.time_limit.seconds = 10
            
            solution = routing.SolveWithParameters(search_parameters)
            if solution:
                routes = extract_routes(manager, routing, solution, num_vehicles)
                total_cost = calculate_total_cost(routes, travel_cost)
                print(f"✓ Solution found with {num_vehicles} vehicles!")
                return routes, total_cost, num_vehicles
        
        return None, None, None
    
    routes_24h, cost_24h, vehicles_24h = solve_with_24h_capacity()
    if routes_24h:
        print(f"Fallback solution: {vehicles_24h} vehicles, cost: €{cost_24h:.2f}")
        with open('1b.txt', 'w') as f:
            for route in routes_24h:
                f.write(' '.join(map(str, route)) + '\n')

Testing helper functions...
Helper functions loaded successfully!

Problem 1b - VRPTW Solution
Total stores to visit: 50
Maximum vehicles to try: 50

Trying with 1 vehicles...
  ✗ Error with 1 vehicles: CP Solver fail

Trying with 2 vehicles...
  ✗ Error with 2 vehicles: CP Solver fail

Trying with 3 vehicles...
  ✗ Error with 3 vehicles: CP Solver fail

Trying with 4 vehicles...
  ✗ Error with 4 vehicles: CP Solver fail

Trying with 5 vehicles...
  ✗ Error with 5 vehicles: CP Solver fail

Trying with 6 vehicles...
  ✗ Error with 6 vehicles: CP Solver fail

Trying with 7 vehicles...
  ✗ Error with 7 vehicles: CP Solver fail

Trying with 8 vehicles...
  ✗ Error with 8 vehicles: CP Solver fail

Trying with 9 vehicles...
  ✗ Error with 9 vehicles: CP Solver fail

Trying with 10 vehicles...
  ✗ Error with 10 vehicles: CP Solver fail

Trying with 11 vehicles...
  ✗ Error with 11 vehicles: CP Solver fail

Trying with 12 vehicles...
  ✗ Error with 12 vehicles: CP Solver fail

Trying with 13 v

##### **b) **

# Problem 1b - VRPTW (multiple salesmen with time windows)
def solve_vrptw_ortools(store_data, travel_time, travel_cost):
    def create_data_model(store_data, travel_time, travel_cost):
        data = {}
        data['time_matrix'] = travel_time.values.astype(int).tolist()
        data['cost_matrix'] = travel_cost.values.astype(int).tolist()
        data['time_windows'] = []
        data['service_time'] = 15
        
        # Add depot time window (24 hours)
        data['time_windows'].append((0, 480))
        
        # Add store time windows
        for _, store in store_data.iterrows():
            open_time = int(store['OpeningTime'])
            close_time = int(store['ClosingTime'])
            data['time_windows'].append((open_time, close_time))
        
        data['num_vehicles'] = 0  # We'll determine this
        data['depot'] = 0
        data['vehicle_capacity'] = 1440
        return data
    
    data = create_data_model(store_data, travel_time, travel_cost)
    max_vehicles = len(store_data)
    
    for num_vehicles in range(1, max_vehicles + 1):
        print(f"Trying with {num_vehicles} vehicles...")
        data['num_vehicles'] = num_vehicles
        
        manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']), data['num_vehicles'], data['depot'])
        routing = pywrapcp.RoutingModel(manager)
        
        def time_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)

            travel_time = data['time_matrix'][from_node][to_node]
            service_time = data['service_time'] if from_node != 0 else 0

            return travel_time + service_time
        
        def cost_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['cost_matrix'][from_node][to_node]
        
        transit_callback_index = routing.RegisterTransitCallback(time_callback)
        cost_callback_index = routing.RegisterTransitCallback(cost_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(cost_callback_index)
        
        # Add time dimension
        routing.AddDimension(
            transit_callback_index,
            1440,
            data['vehicle_capacity'],
            False,
            'Time'
        )
        time_dimension = routing.GetDimensionOrDie('Time')
        
        # Add time window constraints
        for location_idx, time_window in enumerate(data['time_windows']):
            index = manager.NodeToIndex(location_idx)
            time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
        
        # Add service time and disjunctions
        # for location_idx in range(1, len(data['time_windows'])):
        #    index = manager.NodeToIndex(location_idx)
        #    time_dimension.SetCumulVarSoftUpperBound(index, data['time_windows'][location_idx][1], 100000)
        #    routing.AddDisjunction([index], 100000)
        
        # Set depot time windows
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            time_dimension.CumulVar(index).SetRange(
                data['time_windows'][0][0], 
                data['time_windows'][0][1]
                )

            end_index = routing.End(vehicle_id)
            time_dimension.CumulVar(end_index).SetRange(
                data['time_windows'][0][0],
                data['time_windows'][0][1]
            )
        
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        )
        search_parameters.local_search_metaheuristic = (
            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
        )
        search_parameters.time_limit.seconds = 60
        
        solution = routing.SolveWithParameters(search_parameters)
        
        if solution:
            routes = extract_routes(manager, routing, solution, data['num_vehicles'])
            total_cost = calculate_total_cost(routes, travel_cost)
            print(f"Found feasible solution with {num_vehicles} vehicles!")
            return routes, total_cost, num_vehicles
    
    print("No feasible solution found!")
    return None, None, None

# Solve and display results for 1b
print("\n=== Problem 1b - VRPTW Solution ===")
routes_1b, total_cost_1b, num_vehicles_1b = solve_vrptw_ortools(store_data, travel_time, travel_cost)

if routes_1b:
    print(f"\n=== SOLUTION FOUND ===")
    print(f"Number of salesmen needed: {num_vehicles_1b}")
    print(f"Total cost: €{total_cost_1b:.2f}")
    print(f"\nIndividual routes:")
    
    for i, route in enumerate(routes_1b):
        route_time, route_cost = calculate_route_stats(route, travel_time, travel_cost)
        print(f"Salesman {i+1}: {route} (Time: {route_time} min, Cost: €{route_cost:.2f})")
    
    with open('1b.txt', 'w') as f:
        for route in routes_1b:
            f.write(' '.join(map(str, route)) + '\n')